In [1]:

import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw
import IPython
import os

def imshow(img):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret,width=1000,height=1000)
    IPython.display.display(i)


image_folder = "/home/jchamorro/Documents/SISTEMAS_UFF/Images"

# Daliana Swin SAM
predicted_systems_folder = "/home/jchamorro/Documents/SISTEMAS_UFF/Predict_Daliana_systems/Predict_sam"
save_folder = "/home/jchamorro/Documents/SISTEMAS_UFF/predicted_systems_overlay_Daliana_sam"

# Jorge DeeplabV3p
predicted_systems_folder = "/home/jchamorro/Documents/SISTEMAS_UFF/Output_systems/segmentations_systems_color"
save_folder = "/home/jchamorro/Documents/SISTEMAS_UFF/predicted_systems_overlay_Jorge"

# Guedes YoloV8
predicted_systems_folder = "/home/jchamorro/Documents/SISTEMAS_UFF/Predict_Guedes_systems/to_metrics_rgb_Guedes_order"
save_folder = "/home/jchamorro/Documents/SISTEMAS_UFF/predicted_systems_overlay_Guedes"

# Reference

predicted_systems_folder = "/home/jchamorro/Documents/SISTEMAS_UFF/new_labels/Labels_color_bgr2rgb"
save_folder = "/home/jchamorro/Documents/SISTEMAS_UFF/reference_systems_overlay"


# Daliana MAE Imagenet

predicted_systems_folder = "/home/jchamorro/Documents/SISTEMAS_UFF/Predict_Daliana_systems/predictions/Predict_mae_imagenet"
save_folder = "/home/jchamorro/Documents/SISTEMAS_UFF/predicted_systems_overlay_Daliana_mae_imagenet"

# Daliana MAE Sistemas

predicted_systems_folder = "/home/jchamorro/Documents/SISTEMAS_UFF/Predict_Daliana_systems/predictions/Predict_mae_sistemas"
save_folder = "/home/jchamorro/Documents/SISTEMAS_UFF/predicted_systems_overlay_Daliana_mae_sistemas"

# Daliana GT

predicted_systems_folder = "/home/jchamorro/Documents/SISTEMAS_UFF/Predict_Daliana_systems/predictions/Predict_mae_sistemas"
save_folder = "/home/jchamorro/Documents/SISTEMAS_UFF/predicted_systems_overlay_Daliana_mae_sistemas"

'''
# Daliana 360

image_folder = "/home/jchamorro/Documents/PhD/petrobras/swin_docker_image/swin_segmentation/full_images/"
predicted_systems_folder = "/home/jchamorro/Documents/PhD/petrobras/swin_docker_image/result/png"
save_folder = "/home/jchamorro/Documents/PhD/petrobras/swin_docker_image/result/overlay"
'''
# ================ 2024 updated results

# Daliana
predicted_systems_folder = "/home/jchamorro/Documents/PhD/petrobras/resultados_daliana/send_Jorge_2/Predict_cubemap"
save_folder = "/home/jchamorro/Documents/PhD/petrobras/resultados_daliana/send_Jorge_2/predicted_cubemap_overlay"


# Daliana SAM
predicted_systems_folder = "/home/jchamorro/Documents/PhD/petrobras/resultados_daliana/send_Jorge_2/Predict_SAM/"
save_folder = "/home/jchamorro/Documents/PhD/petrobras/resultados_daliana/send_Jorge_2/predicted_SAM_overlay"

# Pedro Soto
predicted_systems_folder = "/home/jchamorro/Documents/petrobras/from_pedro/results_1000_1000/"
save_folder = "/home/jchamorro/Documents/petrobras/from_pedro/results_1000_1000_overlay"

# ====== Experimental set 3 updates

from_colored_predictions_flag = True

results_author = "Daliana" # Daliana, Jorge, Guedes

# colored_value = [255, 0, 255]
colored_value = [150, 5, 61]


outline_flag = False
image_list = ['cubemap_P74_cc39a7d67c298a3ce2a2e077cb322bf3',
'cubemap_P74_0030931c0e3aaaa3a10a9e0f63195eb2',
'cubemap_P74_00c590f7cd08c56d9d73390dd2a085b5',
'cubemap_P74_60e7f7bbe4021bd5e092ba94d6d502b7',
'cubemap_P74_a36a1a90b0a277fdfa89b3e7d934e70f']



print("image_list", image_list)
faces = ['posx', 'posy', 'posz', 'negx', 'negy', 'negz']



outline_width = 5

alpha_value = 77
alpha_value = 90
alpha_value=120
# fill_color = (104, 172, 119, alpha_value)
# outline_color = (104, 172, 119, alpha_value*3)

# fill_color = (255, 255, 255, alpha_value)
# outline_color = (255, 255, 255, alpha_value*3)

fill_color = (255, 0, 255, alpha_value)
outline_color = (255, 0, 255, alpha_value*3)

os.makedirs(save_folder, exist_ok=True)

def set_alpha(img, alpha=120):
    pixdata = img.load()

    width, height = img.size
    for y in range(height):
        for x in range(width):
            if pixdata[x, y] != (0, 0, 0, 255):
                pixdata[x, y] = (pixdata[x, y][0], pixdata[x, y][1], pixdata[x, y][2], alpha)

    for y in range(height):
        for x in range(width):
            if pixdata[x, y] == (0, 0, 0, 255):
                pixdata[x, y] = (0, 0, 0, 0)

    return img

def draw_systems(image_folder, predicted_systems_folder, save_folder, img_name, 
                 input_extension = "JPG", extension = "png", outline_flag = False):

    # ==== Load input image
    image = Image.open(os.path.join(image_folder,f"{img_name}.{input_extension}"))

    # ==== Correct image name
        
    '''
    if from_colored_predictions_flag:
        # img_name_corrected = "predict_total_"+img_name
        img_name_corrected = img_name.split("_")
        img_name_corrected.insert(3, "id")
        img_name_corrected = "_".join(img_name_corrected)

    else:
        img_name_corrected = img_name
    '''
    if results_author == "Daliana":
        img_name_corrected = f"predict_total_{img_name}"
    elif results_author == "Guedes":        
        img_name_corrected = img_name.split("_")
        img_name_corrected.insert(3, "color")
        img_name_corrected = "_".join(img_name_corrected)
    else:
        img_name_corrected = f"{img_name}"

    # print("img_name_corrected", img_name_corrected)       
    
    # ==== Load systems and systems predicted masks
    predicted_systems = Image.open(os.path.join(predicted_systems_folder, f"{img_name_corrected}.{extension}"))
    print("predicted_systems.mode", predicted_systems.mode)
    predicted_systems_npy = np.array(predicted_systems)
    print("predicted_systems_npy.shape", predicted_systems_npy.shape)
    print("predicted_systems.mode", predicted_systems.mode)
    print("predicted_systems_npy.dtype", predicted_systems_npy.dtype)
    '''
    if from_colored_predictions_flag:
        # convert pixels with value colored_value to [255, 255, 255] using np.where and axis=2        
        predicted_systems_npy[np.where((predicted_systems_npy == colored_value).all(axis = 2))] = [255, 255, 255]            
    '''
    # predicted_systems_npy = predicted_systems_npy[:, :, ::-1].copy()
    ## print("0 np.unique(predicted_systems_npy)", np.unique(predicted_systems_npy))
    ## unique_rgba_tuples = np.unique(predicted_systems_npy.reshape(-1, 3), axis=0)
    ## print("1 unique_rgba_tuples", unique_rgba_tuples)

    predicted_systems = Image.fromarray(predicted_systems_npy)

    predicted_systems = predicted_systems.convert("RGBA")
    print("predicted_systems.size", predicted_systems.size)
    print("predicted_systems_npy.shape", predicted_systems_npy.shape)
    '''
    if outline_flag:

        # ==== add border to systems

        plt.imshow(predicted_systems_npy)

        # set an outline around the white pixels

        # predicted_systems_dilate = cv2.dilate(predicted_systems_npy, np.ones((outline_width,outline_width),np.uint8), iterations = 1)
        # predicted_systems_border = cv2.subtract(predicted_systems_dilate, predicted_systems_npy)

        predicted_systems_erode = cv2.erode(predicted_systems_npy, np.ones((outline_width,outline_width),np.uint8), iterations = 1)
        predicted_systems_border = cv2.subtract(predicted_systems_npy, predicted_systems_erode)

        plt.imshow(predicted_systems_border)

        # transform to PIL image
        predicted_systems_border_pil = Image.fromarray(predicted_systems_border)
        predicted_systems_border_pil = predicted_systems_border_pil.convert("RGBA")
        set_alpha(predicted_systems_border_pil, outline_color)
    '''
    ## predicted_systems.save(f"{save_folder}/{img_name}_predicted_systems_before_alpha.png")
    # convert predicted_systems to NPY
    ## tmp = np.array(predicted_systems)
    ## unique_rgba_tuples = np.unique(tmp.reshape(-1, 4), axis=0)
    ## print("2 unique_rgba_tuples", unique_rgba_tuples)

    # ==== Set fill color with transparency
    set_alpha(predicted_systems)

    # image_np_RGB = np.array(image)

    # image_np_BGR = np.flip(image_np_RGB ,-1)
    # image = Image.fromarray(image_np_BGR)
    image = image.convert("RGBA")
    # ==== Merge systems with input image

    ## image.save(f"{save_folder}/{img_name}_before_composite.png")
    ## predicted_systems.save(f"{save_folder}/{img_name}_predicted_systems.png")

    annotated_face = Image.alpha_composite(image, predicted_systems)
    '''
    if outline_flag:
        annotated_face = Image.alpha_composite(annotated_face, predicted_systems_border_pil)
    '''
    # ==== Save image
    annotated_face.save(f"{save_folder}/{img_name}.png")




image_list ['cubemap_P74_cc39a7d67c298a3ce2a2e077cb322bf3', 'cubemap_P74_0030931c0e3aaaa3a10a9e0f63195eb2', 'cubemap_P74_00c590f7cd08c56d9d73390dd2a085b5', 'cubemap_P74_60e7f7bbe4021bd5e092ba94d6d502b7', 'cubemap_P74_a36a1a90b0a277fdfa89b3e7d934e70f']


In [2]:

# iterate over all images in image_folder without extension
for image_name in image_list:
    for face in faces:
        img_name = f"{image_name}_{face}"
        if img_name.endswith(".png"):
            img_name = img_name.replace(".png", "")
        print("img_name", img_name)
        draw_systems(image_folder, predicted_systems_folder, save_folder, img_name, input_extension="png", extension="png")


img_name cubemap_P74_cc39a7d67c298a3ce2a2e077cb322bf3_posx
predicted_systems.mode RGB
predicted_systems_npy.shape (1344, 1344, 3)
predicted_systems.mode RGB
predicted_systems_npy.dtype uint8
predicted_systems.size (1344, 1344)
predicted_systems_npy.shape (1344, 1344, 3)
img_name cubemap_P74_cc39a7d67c298a3ce2a2e077cb322bf3_posy
predicted_systems.mode RGB
predicted_systems_npy.shape (1344, 1344, 3)
predicted_systems.mode RGB
predicted_systems_npy.dtype uint8
predicted_systems.size (1344, 1344)
predicted_systems_npy.shape (1344, 1344, 3)
img_name cubemap_P74_cc39a7d67c298a3ce2a2e077cb322bf3_posz
predicted_systems.mode RGB
predicted_systems_npy.shape (1344, 1344, 3)
predicted_systems.mode RGB
predicted_systems_npy.dtype uint8
predicted_systems.size (1344, 1344)
predicted_systems_npy.shape (1344, 1344, 3)
img_name cubemap_P74_cc39a7d67c298a3ce2a2e077cb322bf3_negx
predicted_systems.mode RGB
predicted_systems_npy.shape (1344, 1344, 3)
predicted_systems.mode RGB
predicted_systems_npy.dtype ui

In [3]:
'''
import glob
extension = "JPG"
# iterate over all images in image_folder without extension
folder_path = os.path.join(image_folder, f"*.{extension}")
print("folder_path", folder_path)
for path in glob.glob(folder_path):
    img_name = os.path.basename(path)
    if img_name.endswith(f".{extension}"):
        img_name = img_name.replace(f".{extension}", "")
    print("img_name", img_name)
    draw_systems(image_folder, predicted_systems_folder, save_folder, img_name, input_extension="JPG", extension="png")
'''

'\nimport glob\nextension = "JPG"\n# iterate over all images in image_folder without extension\nfolder_path = os.path.join(image_folder, f"*.{extension}")\nprint("folder_path", folder_path)\nfor path in glob.glob(folder_path):\n    img_name = os.path.basename(path)\n    if img_name.endswith(f".{extension}"):\n        img_name = img_name.replace(f".{extension}", "")\n    print("img_name", img_name)\n    draw_systems(image_folder, predicted_systems_folder, save_folder, img_name, input_extension="JPG", extension="png")\n'